In [ ]:
import torch


# Autograd: Dérivation automatique

Grâce à `torch.autograd`, PyTorch permet d'**enregistrer "on the fly" le graphe** d'une suite d'opérations. Cela est utile pour **calculer le gradient** (dérivées partielles) d'une fonction par rapport à certains de ses paramètres.

Ce qui distingue PyTorch (au moment de son apparition), c'est qu'il fonctionne avec le principe "**define-by-run**"; c'est-à-dire qu'au lieu que les graphes soient compilés à l'avance comme dans TensorFlow, les graphes sont définis à mesure que le code Python est éxécuté. Cela apporte une grande flexibilité et une facilité pour le prototypage.

## Tensor

`torch.Tensor` est la classe centrale de `torch.autograd`. Pour enregistrer toutes les opérations effectuées sur un tensor, il faut mettre sont attribut `.requires_grad` à `True`.

Voyons un exemple. Supposons que nous voulons entraîner une régression linéaire de la forme `y = x @ w`, où `@` est une multiplication matricielle.

In [ ]:
x = torch.rand(2, 2)       # Données en entrée
y_true = torch.rand(2)  # Données cible

Créons le vecteur de poids `w` à entraîner.

In [ ]:
w = torch.rand(2, requires_grad=True)
print(w)

Maintenant, effectuons une inférence:

In [ ]:
y_pred = x @ w
print(y_pred)

`y_pred` est le résultat d'une multiplication matricielle. Il a donc un attribut `grad_fn` qui référencie la fonction utilisée:

In [ ]:
print(y_pred.grad_fn)

Calculons maintenant l'erreur quadratique sur ces prédictions:

In [ ]:
loss_per_element = (y_true - y_pred) ** 2
loss = loss_per_element.mean()

print(loss_per_element)
print(loss)

## Backpropagation des gradients

Nous allons maintenant effectuer une "backpropagation" afin d'obtenir la dérivée de la loss par rapport aux paramètres:

In [ ]:
loss.backward()

PyTorch parcourt le graphe et place les valeurs de dérivées dans l'attribut `.grad` des tenseurs ayant demandé le calcul du leur gradient. Affichons le gradient de l'erreur par rapport à `w`:

In [ ]:
print(w.grad)

Grâce à ces dérivées partielles, nous pouvons maintenant faire une descente de gradient afin d'optimiser `w`:

In [ ]:
w.data -= w.grad

Faisons maintenant une nouvelle inférence, et calculons l'erreur:

In [ ]:
old_loss = loss.item()

y_pred = x @ w
loss = ((y_true - y_pred) ** 2).mean() 

print('Avant', old_loss)
print('Apres', loss.item())

Remarquez que l'erreur a diminué! Magie!